In [1]:
!pip install -q -U kaggle_environments

In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Loading environment lux_ai_2022 failed: No module named 'vec_noise'


In [3]:
'''
Вывод по агентам в конце!!!

'''

'\nСтратегии для агентов:\n1) Только камень\n2) Только бумага\n3) Только ножницы\n4) Рандом\n\n5) Только камень, а после то, что использовал оппонент \n6) Только ножницы, а после то, что использовал оппонент\n7) Только бумага, а после то, что использовал оппонент\n8) Рандом, а после то, что использовал оппонент\n\n9) Камень, если оппонент использовал камень то бумагу в следующем раунде\n10) Бумага, если оппонент использовал бумагу то ножницы в следующем раунде\n11) Ножницы, если оппонент использовал ножницы то камень в следующем раунде\n12) Рандом, если оппонент использовал ножницы то камень в следующем раунде\n\n'

In [4]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    #Только камень
    return 0

Overwriting rock_agent.py


In [5]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    #Только бумага
    return 1

Overwriting paper_agent.py


In [6]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    #Только ножницы
    return 2

Overwriting scissors_agent.py


In [7]:
%%writefile random_agent.py

def random_agent(observation, configuration):
    #Только random
    import random
    return random.randrange(0, 3)

Overwriting random_agent.py


In [69]:
%%writefile copy_opponent.py

#Example 
def copy_opponent(observation, configuration):
    import random
    #Копирует предыдущий шаг оппонента
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [11]:
%%writefile statistical.py

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs


Overwriting statistical.py


In [12]:
%%writefile shuffle_agent.py
shuffle_count = 0
def shuffle_agent(observation, configuration):
    #Перебирает варианты боя по очереди, начиная с камня
    global shuffle_count
    if shuffle_count == 0:
        shuffle_count += 1
        return 0
    elif shuffle_count == 1:
        shuffle_count += 1
        return 1
    elif shuffle_count == 2:
        shuffle_count = 0
        return 2
            

Overwriting shuffle_agent.py


In [13]:
%%writefile oppose_agent.py

#Example 
def oppose_agent(observation, configuration):
    #Oтличное от оппонента
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return random.choice([1,2])
        elif observation.lastOpponentAction == 1:
            return random.choice([0,2])
        elif observation.lastOpponentAction == 2:
            return random.choice([0,1])
    else:
        return random.randrange(0, 3)

Overwriting oppose_agent.py


In [21]:
%%writefile anti_shuffle_agent.py

#Example 
def anti_shuffle_agent(observation, configuration):
    #Стратегия чтобы побить shuffle_agent.py
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 2
        elif observation.lastOpponentAction == 1:
            return 0
        elif observation.lastOpponentAction == 2:
            return 1
    else:
        return 1

Writing anti_shuffle_agent.py


In [23]:
%%writefile reverse_shuffle_agent.py
reverse_shuffle_count = 2
def shuffle_agent(observation, configuration):
    #Перебирает варианты боя в обратной очереди, начиная с ножниц
    global reverse_shuffle_count
    if reverse_shuffle_count == 2:
        reverse_shuffle_count -= 1
        return 2
    elif reverse_shuffle_count == 1:
        reverse_shuffle_count -= 1
        return 1
    elif reverse_shuffle_count == 0:
        reverse_shuffle_count += 2
        return 0

Writing reverse_shuffle_agent.py


In [27]:
%%writefile alt_random_agent.py

def alt_random_agent(observation, configuration):
    #Альтернативная реализация рандома через метод choice
    import random
    return random.choice([0,1,2])

Writing alt_random_agent.py


In [30]:
%%writefile loser_agent.py

#Example 
def loser_agent(observation, configuration):
    #Попытка проиграть специально
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 2
        elif observation.lastOpponentAction == 1:
            return 0
        elif observation.lastOpponentAction == 2:
            return 1
    else:
        return random.choice([0,1,2])

Writing loser_agent.py


In [67]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
rock_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        rock_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[0], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(rock_agent_res)
print(len(rock_agent_res))

[0, -999.0, 999.0, 0, 997.0, 1, 999.0, 51.0, 0, 0, 0, -998.0]
12


In [66]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
paper_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        paper_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[1], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(paper_agent_res)
print(len(paper_agent_res))

[999.0, 0, -999.0, 38.0, 998.0, 1, 998.0, 0, 0, 0, 0, -997.0]
12


In [65]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
scissors_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        scissors_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[2], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(scissors_agent_res)
print(len(scissors_agent_res))

[-999.0, 999.0, 0, -22.0, 999.0, 1, 998.0, 21.0, 0, 0, 0, -999.0]
12


In [64]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
alt_random_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        alt_random_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[3], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(alt_random_agent_res)
print(len(alt_random_agent_res))

[0, -40.0, 0, -37.0, 0, 1, 24.0, 0, -43.0, -34.0, -47.0, 29.0]
12


In [63]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
anti_shuffle_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        anti_shuffle_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[4], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(anti_shuffle_agent_res)
print(len(anti_shuffle_agent_res))

[-997.0, -998.0, -999.0, 0, 0, 1, 0, 37.0, 35.0, 0, 999.0, -996.0]
12


In [70]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
copy_opponent_res = []
for i in range(len(agent_list)):
    if i < 12:
        copy_opponent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[5], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(copy_opponent_res)
print(len(copy_opponent_res))

[0, 0, 0, 0, 500.0, 0, 500.0, 0, 0, 998.0, -999.0, -200.0]
12


In [71]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
loser_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        loser_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[6], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(loser_agent_res)
print(len(loser_agent_res))

[-998.0, -998.0, -999.0, 22.0, 0, 999.0, 0, 0, 0, 0, 997.0, -996.0]
12


In [72]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
oppose_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        oppose_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[7], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(oppose_agent_res)
print(len(oppose_agent_res))

[0, 0, 42.0, -47.0, -21.0, 0, 0, -27.0, 0, -489.0, 478.0, 0]
12


In [73]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
random_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        random_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[8], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(random_agent_res)
print(len(random_agent_res))

[0, 0, 46.0, -52.0, 0, -29.0, 0, -20.0, 0, 0, 35.0, 0]
12


In [74]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
reverse_shuffle_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        reverse_shuffle_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[9], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(reverse_shuffle_agent_res)
print(len(reverse_shuffle_agent_res))

[0, 0, 0, 38.0, 0, -999.0, 0, 485.0, 0, 0, 0, 0]
12


In [75]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
shuffle_agent_res = []
for i in range(len(agent_list)):
    if i < 12:
        shuffle_agent_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[10], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(shuffle_agent_res)
print(len(shuffle_agent_res))

[0, 0, 0, 29.0, -999.0, 997.0, -997.0, -493.0, 0, 0, 0, 0]
12


In [76]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'alt_random_agent.py', 'anti_shuffle_agent.py',
             'copy_opponent.py', 'loser_agent.py', 'oppose_agent.py', 'random_agent.py', 'reverse_shuffle_agent.py',
             'shuffle_agent.py', 'statistical.py']
statistical_res = []
for i in range(len(agent_list)):
    if i < 12:
        statistical_res.append(evaluate(
        "rps", #environment to use - no need to change
        [agent_list[11], agent_list[0+i]], #agents to evaluate
        configuration={"episodeSteps": 1000} #number of episodes 
        )[0][0])
    else:
        break
print(statistical_res)
print(len(statistical_res))

[998.0, 997.0, 999.0, 31.0, 996.0, 200.0, 999.0, 0, 0, 0, 0, 0]
12


In [ ]:
###
Лучше всего себя показал статистический агент, который учитывает предыдущие шаги оппонента на дистанции.
Рандомные агенты (random_agent.py, alt_random_agent.py) показали себя вполне неплохо на длинной дистанции.
Хуже всего себя показал loser_agent.py ведь он специально создан для того чтобы давать наиболее удобные условия оппоненту,
однако он не справился с двумя агентами shuffle_agent.py и reverse_agent.py которые полностью игнорируют действия оппонента и
выдают камень, ножницы и бумагу по очереди, с разницей лишь в порядке.
###